We have a mesh, consisting of vertices and corresponding faces.

In [1]:
import openmesh as om
import numpy as np
import matplotlib.pyplot as plt

mesh = om.read_trimesh('T4.off')
V=mesh.points()
X=V[:,0]
Y=V[:,1]

Get the nodes at the incoming boundary. We register them in an active node list ActiveNodeList. (Or "Calculated node list" ?) The purpose is to get all the other nodes also into this list.

In [2]:
def initActiveNodes():
    ActiveNodeList = []
    for vh in mesh.vertices():
        k = vh.idx()
        if V[k,0] == 0 or V[k,1] == 1:
            ActiveNodeList.append(vh)
    return ActiveNodeList

ActiveNodeList = initActiveNodes()
            
for vh in ActiveNodeList:
    print(vh.idx())


0
5
10
15
20
25
26
27
28
29


Then we have the list of active faces, i.e. faces that are in the queue to calculate the unknowns in the third node, from values given for the other nodes. If all three values in a face are calculated then the corresponding face passes to the list of treated faces, eventually making place for a next face to incorporate.

We run over all faces and count for each face the number of active nodes. If there are two active nodes in a face then that face is agregated to the list of active faces.

In [3]:

def initActiveFaceList():
    ActiveFaceList = []

    i=0
    for fh in mesh.faces():
        counter = 0
    
        for vh in mesh.fv(fh):
            if vh in ActiveNodeList:
                print(vh.idx(),' in ', fh.idx())
                counter = counter + 1
            
        if counter == 2:
            ActiveFaceList.append(fh)

    return ActiveFaceList

ActiveFaceList = initActiveFaceList()



0  in  0
26  in  1
25  in  1
28  in  4
29  in  4
5  in  5
0  in  5
10  in  7
5  in  8
10  in  8
5  in  9
15  in  11
10  in  11
28  in  22
27  in  22
26  in  35
26  in  40
27  in  40
26  in  42
15  in  43
29  in  47
27  in  52
27  in  55
28  in  57
28  in  60
20  in  74
25  in  74
20  in  75
15  in  75
20  in  76


Checking whether the generated gives the correct values of the sample mesh, it seems that the active faces are identified correctly.

In [4]:
for fh in ActiveFaceList:
    print(fh.idx())

1
4
5
8
11
22
40
74
75


We need to assign the boundary condition at these initially active nodes. Eventually the nodes should be organized within a class as data structure, where each node has corresponding variables assigned. Alternatively we create an array of the same length of the total node number.



In [5]:
def init_TS():
    T = np.zeros(len(V))+7
    S = np.zeros(len(V))+7

    for vh in ActiveNodeList:    
        k = vh.idx()
        if V[k,0] == 0:
            T[k] = 0
            S[k] = 0
        elif V[k,1] == 1:
            T[k] = V[k,0]
            S[k] = 0
    
    return T, S



Some display of the variables, that might be needed for a later debugging.

In [6]:
T, S = init_TS()
I = range(len(V))
Z=np.vstack([I,T,S])
Z.transpose()

array([[ 0.,  0.,  0.],
       [ 1.,  7.,  7.],
       [ 2.,  7.,  7.],
       [ 3.,  7.,  7.],
       [ 4.,  7.,  7.],
       [ 5.,  0.,  0.],
       [ 6.,  7.,  7.],
       [ 7.,  7.,  7.],
       [ 8.,  7.,  7.],
       [ 9.,  7.,  7.],
       [10.,  0.,  0.],
       [11.,  7.,  7.],
       [12.,  7.,  7.],
       [13.,  7.,  7.],
       [14.,  7.,  7.],
       [15.,  0.,  0.],
       [16.,  7.,  7.],
       [17.,  7.,  7.],
       [18.,  7.,  7.],
       [19.,  7.,  7.],
       [20.,  0.,  0.],
       [21.,  7.,  7.],
       [22.,  7.,  7.],
       [23.,  7.,  7.],
       [24.,  7.,  7.],
       [25.,  0.,  0.],
       [26.,  1.,  0.],
       [27.,  2.,  0.],
       [28.,  3.,  0.],
       [29.,  4.,  0.],
       [30.,  7.,  7.],
       [31.,  7.,  7.],
       [32.,  7.,  7.],
       [33.,  7.,  7.],
       [34.,  7.,  7.],
       [35.,  7.,  7.],
       [36.,  7.,  7.],
       [37.,  7.,  7.],
       [38.,  7.,  7.],
       [39.,  7.,  7.],
       [40.,  7.,  7.],
       [41.,  7.

Each face should have a priority of handling, which corresponds to the assigned time in terms
of the averaged $T$ - values of the two incoming nodes
\begin{align*}
    T_{ABC} = \frac{T_A + T_B}{2}
\end{align*}
Therefore we generate arrays that contain predictive $T$ and $S$ values for each face. The values are predictive as they are replaced with calculated values as soon as the equations of the corresponding face are solved.

In [7]:

Tf = np.zeros(len(mesh.faces()))+7
Sf = np.zeros(len(mesh.faces()))+7

for fh in ActiveFaceList:
    T_ABC = 0
    S_ABC = 0
    counter = 0
    for vh in mesh.fv(fh):
        if vh in ActiveNodeList:
            T_ABC = T_ABC + T[vh.idx()]
            S_ABC = S_ABC + S[vh.idx()]            
            counter = counter + 1
            
    if counter != 2:
        print('ERR at face', fh.idx())
    else:
        Tf[fh.idx()] = T_ABC / 2
        Sf[fh.idx()] = S_ABC / 2
        
        

Some output for an eventual manual checking of values, eventually lateron as the algorithm proceeds.

In [8]:
I = range(len(Tf))
Z=np.vstack([I,Tf,Sf])
Z.transpose()

array([[ 0. ,  7. ,  7. ],
       [ 1. ,  0.5,  0. ],
       [ 2. ,  7. ,  7. ],
       [ 3. ,  7. ,  7. ],
       [ 4. ,  3.5,  0. ],
       [ 5. ,  0. ,  0. ],
       [ 6. ,  7. ,  7. ],
       [ 7. ,  7. ,  7. ],
       [ 8. ,  0. ,  0. ],
       [ 9. ,  7. ,  7. ],
       [10. ,  7. ,  7. ],
       [11. ,  0. ,  0. ],
       [12. ,  7. ,  7. ],
       [13. ,  7. ,  7. ],
       [14. ,  7. ,  7. ],
       [15. ,  7. ,  7. ],
       [16. ,  7. ,  7. ],
       [17. ,  7. ,  7. ],
       [18. ,  7. ,  7. ],
       [19. ,  7. ,  7. ],
       [20. ,  7. ,  7. ],
       [21. ,  7. ,  7. ],
       [22. ,  2.5,  0. ],
       [23. ,  7. ,  7. ],
       [24. ,  7. ,  7. ],
       [25. ,  7. ,  7. ],
       [26. ,  7. ,  7. ],
       [27. ,  7. ,  7. ],
       [28. ,  7. ,  7. ],
       [29. ,  7. ,  7. ],
       [30. ,  7. ,  7. ],
       [31. ,  7. ,  7. ],
       [32. ,  7. ,  7. ],
       [33. ,  7. ,  7. ],
       [34. ,  7. ,  7. ],
       [35. ,  7. ,  7. ],
       [36. ,  7. ,  7. ],
 

The purpose of this asignation of values to each face is to know which is next to deal with. So, the indices are sorted to access to increasing $T$ values, which permits an access to the smallest values.

In [9]:
If = np.argsort(Tf, axis=0)
Z = np.vstack([If,Tf[If],Sf[If]])
Z.transpose()

array([[ 5. ,  0. ,  0. ],
       [75. ,  0. ,  0. ],
       [74. ,  0. ,  0. ],
       [ 8. ,  0. ,  0. ],
       [11. ,  0. ,  0. ],
       [ 1. ,  0.5,  0. ],
       [40. ,  1.5,  0. ],
       [22. ,  2.5,  0. ],
       [ 4. ,  3.5,  0. ],
       [ 0. ,  7. ,  7. ],
       [57. ,  7. ,  7. ],
       [56. ,  7. ,  7. ],
       [55. ,  7. ,  7. ],
       [54. ,  7. ,  7. ],
       [53. ,  7. ,  7. ],
       [52. ,  7. ,  7. ],
       [48. ,  7. ,  7. ],
       [50. ,  7. ,  7. ],
       [49. ,  7. ,  7. ],
       [58. ,  7. ,  7. ],
       [47. ,  7. ,  7. ],
       [46. ,  7. ,  7. ],
       [45. ,  7. ,  7. ],
       [51. ,  7. ,  7. ],
       [59. ,  7. ,  7. ],
       [62. ,  7. ,  7. ],
       [61. ,  7. ,  7. ],
       [77. ,  7. ,  7. ],
       [76. ,  7. ,  7. ],
       [73. ,  7. ,  7. ],
       [72. ,  7. ,  7. ],
       [71. ,  7. ,  7. ],
       [70. ,  7. ,  7. ],
       [60. ,  7. ,  7. ],
       [69. ,  7. ,  7. ],
       [67. ,  7. ,  7. ],
       [66. ,  7. ,  7. ],
 

Actually our focus is on the active faces, so we need to get the indices of them.

In [10]:
def getActiveFaceIndices():
    I_list = []
    for fh in ActiveFaceList:
        I_list.append(fh.idx())
        
    return I_list

def printActiveFaceIndices():
    I_active_faces = getActiveFaceIndices()
    print('active faces:', I_active_faces)
    
printActiveFaceIndices()


active faces: [1, 4, 5, 8, 11, 22, 40, 74, 75]


We can focus the sorting of who comes next within this list of active faces.

So, taking the list of active faces, we sort the elements within this list, by an internal permutation index $K$, such that the active elements on the global list are identified in their identified order by $I[K]$.


In [11]:
I_active_faces = getActiveFaceIndices()
I = np.array(I_active_faces)
K = np.argsort(Tf[I], axis=0)

Z = np.vstack([I[K],Tf[I[K]],Sf[I[K]]])
Z.transpose()

array([[ 5. ,  0. ,  0. ],
       [ 8. ,  0. ,  0. ],
       [11. ,  0. ,  0. ],
       [74. ,  0. ,  0. ],
       [75. ,  0. ,  0. ],
       [ 1. ,  0.5,  0. ],
       [40. ,  1.5,  0. ],
       [22. ,  2.5,  0. ],
       [ 4. ,  3.5,  0. ]])

Now we are in the starting position of the iteration that goes through all faces. Taking the next face, the values of the remaining node are calculated. In a dummy calculation one might just set a value like
\begin{align*}
    T_{C} = \frac{T_A + T_B}{2} + 1 .
\end{align*}
Such a dummy calculation helps traverse the overall mesh focusing on the propagation mecanism.


In [12]:
def get_T_C_dummy(T_A, T_B):
        
    return (T_A + T_B)/2+1
    
T_C = get_T_C_dummy(3.3, 5.5)
print(T_C)

5.4


So, the procedure in a loop cycle is to (1) obtain the id of the next face, (2) calculate the value of the outgoing vertex, and (3) handle the list of active faces, i. e. (3a) remove the processed face from the list, and (3b) agregate new faces to the list. These new faces are those faces at the now new ingoing nodes, as long as these faces are not yet processed or are not already in the list. Eventually, one might want to access to the ingoing and outgoing nodes of a faces by a corresponding data structure. Otherwise one need to identify the node characteristics inline.

So, a first step is distinguish the ingoing and outgoing nodes of an active face.

In [13]:
ProcessedFaceList = []

# (1) obtain id of next face
next_face_id = (I[K])[0]
print('next face id:', next_face_id)
fh = mesh.face_handle(next_face_id)

# (2) calculate value of outgoing vertex
counter = 0
for vh in mesh.fv(fh):
    if vh in ActiveNodeList:
        print(vh.idx(),'ingoing')
        if counter == 0:
            T_A = T[vh.idx()]
        elif counter == 1:
            T_B = T[vh.idx()]
        else:
            print('ERR: too many ingoing nodes')
            
        counter = counter + 1
    else:
        print(vh.idx(),'outgoing')        
        id_C = vh.idx()
        
if counter == 2:
    T_C = get_T_C_dummy(T_A, T_B)
else:
    print('ERR: missing inputs for calculating T_C')
    
    
# ActiveFaceList.remove(fh)
ProcessedFaceList.append(fh)

printActiveFaceIndices()
for ffh in mesh.ff(fh):
    if (not ffh in ActiveFaceList) and (not ffh in ProcessedFaceList):
        # print(ffh.idx())
        ActiveFaceList.append(ffh)


next face id: 5
30 outgoing
5 ingoing
0 ingoing
active faces: [1, 4, 5, 8, 11, 22, 40, 74, 75]


One exercise is to traverse the overall mesh just by starting with the face, and iteratively passing through the next faces, without recurring to $T-$order criterium.

In [14]:
while len(ActiveFaceList)>0:
    I_active = getActiveFaceIndices()
    print(len(I_active),': ',I_active)
    next_face_id = I_active[0]
    fh = mesh.face_handle(next_face_id)
    
    ActiveFaceList.remove(fh)
    ProcessedFaceList.append(fh)

    # printActiveFaceIndices()
    for ffh in mesh.ff(fh):
        if (not ffh in ActiveFaceList) and (not ffh in ProcessedFaceList):
            ActiveFaceList.append(ffh)

11 :  [1, 4, 5, 8, 11, 22, 40, 74, 75, 0, 9]
11 :  [4, 5, 8, 11, 22, 40, 74, 75, 0, 9, 35]
12 :  [5, 8, 11, 22, 40, 74, 75, 0, 9, 35, 60, 47]
11 :  [8, 11, 22, 40, 74, 75, 0, 9, 35, 60, 47]
11 :  [11, 22, 40, 74, 75, 0, 9, 35, 60, 47, 7]
11 :  [22, 40, 74, 75, 0, 9, 35, 60, 47, 7, 43]
12 :  [40, 74, 75, 0, 9, 35, 60, 47, 7, 43, 52, 57]
13 :  [74, 75, 0, 9, 35, 60, 47, 7, 43, 52, 57, 42, 55]
13 :  [75, 0, 9, 35, 60, 47, 7, 43, 52, 57, 42, 55, 76]
12 :  [0, 9, 35, 60, 47, 7, 43, 52, 57, 42, 55, 76]
12 :  [9, 35, 60, 47, 7, 43, 52, 57, 42, 55, 76, 32]
12 :  [35, 60, 47, 7, 43, 52, 57, 42, 55, 76, 32, 33]
12 :  [60, 47, 7, 43, 52, 57, 42, 55, 76, 32, 33, 44]
12 :  [47, 7, 43, 52, 57, 42, 55, 76, 32, 33, 44, 59]
12 :  [7, 43, 52, 57, 42, 55, 76, 32, 33, 44, 59, 48]
12 :  [43, 52, 57, 42, 55, 76, 32, 33, 44, 59, 48, 6]
12 :  [52, 57, 42, 55, 76, 32, 33, 44, 59, 48, 6, 73]
12 :  [57, 42, 55, 76, 32, 33, 44, 59, 48, 6, 73, 51]
12 :  [42, 55, 76, 32, 33, 44, 59, 48, 6, 73, 51, 56]
12 :  [55, 76

This procedure can be represented also by the list of processed faces, which was generated all over the process.

In [15]:
def getProcessedFaceIndices():
    I_list = []
    for fh in ProcessedFaceList:
        I_list.append(fh.idx())
        
    return I_list

def printProcessedFaceIndices():
    I_faces = getProcessedFaceIndices()
    print('processed faces:', I_faces)
    
printProcessedFaceIndices()   

I_faces = getProcessedFaceIndices()
print('check completeness:', np.sort(I_faces))

processed faces: [5, 1, 4, 5, 8, 11, 22, 40, 74, 75, 0, 9, 35, 60, 47, 7, 43, 52, 57, 42, 55, 76, 32, 33, 44, 59, 48, 6, 73, 51, 56, 41, 54, 31, 34, 72, 61, 49, 71, 50, 58, 68, 53, 10, 29, 30, 24, 21, 70, 66, 62, 69, 38, 16, 15, 36, 23, 45, 65, 63, 39, 13, 14, 37, 12, 46, 67, 64, 3, 28, 27, 20, 79, 25, 17, 26, 19, 78, 18, 77, 2]
check completeness: [ 0  1  2  3  4  5  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22
 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46
 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70
 71 72 73 74 75 76 77 78 79]


## Class of nodes
We define a class of nodes that represent the main geometrical components.


## List of faces
We operate on a list of faces that track, that priorizes in which direction to advances, by evaluating (1) whether already two points are calculated and (2) whether the causality condition is satisfied.

It also reports whether the overall procedure gets stuckº

## Managing lists

In [16]:
myList = [2, 4, 10, 19, 9];
myList.remove(10);
myList.append(5)
print(myList);

[2, 4, 19, 9, 5]
